In [90]:
from typing import Dict, List

class Solution:
    def memoHelper(self, prices: List[int], i: int, has_stock: bool, limit: int, memo: Dict[int, int]) -> int:
        if i == len(prices) or limit == 0:
            return 0
        # if we currently have the stock, we can either sell it for the current price or wait for the next day
        # if we don't have the stock, we can either buy it today or wait for the next day
        if has_stock:
            if (i, has_stock, limit) not in memo:
                memo[(i, has_stock, limit)] = max(self.memoHelper(prices, i+1, False, limit-1, memo) 
                    + prices[i], self.memoHelper(prices, i+1, True, limit, memo))
            return memo[(i, has_stock, limit)]
        else:
            if (i, has_stock, limit) not in memo:
                memo[(i, has_stock, limit)] = max(self.memoHelper(prices, i+1, True, limit -1, memo) 
                    - prices[i], self.memoHelper(prices, i+1, False, limit, memo))
            return memo[(i, has_stock, limit)]
        
    # uses recursion and memoization to solve the problem
    # Time complexity of ~ O(nk) where n is the number of days and k is the 
    # maximum number of transactions
    def maxProfitMemo(self, k: int, prices: List[int]) -> int:
        memo = {}
        return self.memoHelper(prices, 0, False, 2*k, memo)
    
    def maxProfitRecursive(self, k: int, prices: List[int]) -> int:
        def helper(i: int, has_stock: bool, limit: int) -> int:
            if i == len(prices) or limit == 0:
                return 0
            if has_stock:
                return max(helper(i+1, False, limit-1) + prices[i], helper(i+1, True, limit))
            else:
                return max(helper(i+1, True, limit-1) - prices[i], helper(i+1, False, limit))
            
        return helper(0, False, 2*k)
    
    def maxProfitIterative(self, k: int, prices: List[int]) -> int:
        stack = [(0, False, 0, 2*k)] # (i, has_stock, profit, transactions)
        max_profit = 0

        while stack:
            i, state, profit, t = stack.pop()
            if i == len(prices) or t == 0:
                max_profit = max(max_profit, profit)
                continue
            if state:
                stack.append((i+1, False, profit + prices[i], t-1))
                stack.append((i+1, True, profit, t))
            else:
                stack.append((i+1, True, profit - prices[i], t-1))
                stack.append((i+1, False, profit, t))
                
        return max_profit
    
    def maxProfitDP(self, k: int, prices: List[int]) -> int:
        if not prices:
            return 0

        n = len(prices)
        dp = [[[0 for _ in range(2*k + 1)] for _ in range(2)] for _ in range(n + 1)]

        # Fill the DP table bottom-up
        for i in range(n - 1, -1, -1):
            for has_stock in range(2):
                for t in range(2*k + 1):
                    if t == 0:
                        dp[i][has_stock][t] = 0
                        continue

                    if has_stock:
                        # Max of selling the stock or holding
                        dp[i][has_stock][t] = max(dp[i+1][0][t-1] + prices[i], dp[i+1][1][t])
                    else:
                        # Max of buying the stock or not buying
                        dp[i][has_stock][t] = max(dp[i+1][1][t-1] - prices[i], dp[i+1][0][t])

        return dp[0][0][2*k]
    
    # don't need to keep track of the days since we only need the next day's values
    def maxProfit(self, k: int, prices: List[int]) -> int:
        if not prices:
            return 0

        n = len(prices)
        dp = [[0 for _ in range(2*k + 1)] for _ in range(2)]

        # Fill the DP table bottom-up
        for i in range(n - 1, -1, -1):
            for has_stock in range(2):
                for t in range(2*k + 1):
                    if t == 0:
                        dp[has_stock][t] = 0
                        continue

                    if has_stock:
                        # Max of selling the stock or holding
                        dp[has_stock][t] = max(dp[0][t-1] + prices[i], dp[1][t])
                    else:
                        # Max of buying the stock or not buying
                        dp[has_stock][t] = max(dp[1][t-1] - prices[i], dp[0][t])

        return dp[0][2*k]
    

In [91]:
import time
import random
import sys

random.seed(0)
sys.setrecursionlimit(10**8)

soln = Solution()
f = soln.maxProfitDP
# test 0
k = 4
prices = [1,2,3,4,5,6,7,8,9,10,1,10,1,10,1,10]
print("max profit:", f(k, prices), "expected:", 36)

# test 1
k = 2
prices = [2,4,1]
print("max profit:", f(k, prices), "expected:", 2) 

# test 2
k = 2
prices = [3,2,6,5,0,3]
print("max profit:", f(k, prices), "expected:", 7)

# test 3
k = 4
prices = [7,6,4,3,1]
print("max profit:", f(k, prices), "expected:", 0)

# # tests for determining algorithm complexity
# k = 100

# test 4
n = 10
prices = [random.randint(1, n) for _ in range(n)]
start = time.time()
print("profit:", f(k, prices), "expected:", 11)
print("  time:", time.time()-start)

# test 5
n = 100
prices = [random.randint(1, n) for _ in range(n)]
start = time.time()
print("profit:", f(k, prices,), "expected:", 358)
print("  time:", time.time()-start)

# # test 6
# n = 1000
# prices = [random.randint(1, n) for _ in range(n)]
# start = time.time()
# print("profit:", f(k, prices), "expected:", 3978)
# print("  time:", time.time()-start)

# # test 7
# n = 10000
# prices = [random.randint(1, n) for _ in range(n)]
# start = time.time()
# print("profit:", f(k, prices), "expected:", 39975)
# print("  time:", time.time()-start)

# # test 8
# n = 100000
# prices = [random.randint(1, n) for _ in range(n)]
# start = time.time()
# print("profit:", f(k, prices), "expected:", 399968)
# print("  time:", time.time()-start)

# # test 9
# n = 1000000
# prices = [random.randint(1, n) for _ in range(n)]
# start = time.time()
# print("profit:", f(k, prices), "expected:", 3999971)
# print("  time:", time.time()-start)

max profit: 36 expected: 36
max profit: 2 expected: 2
max profit: 7 expected: 7
max profit: 0 expected: 0
profit: 11 expected: 11
  time: 0.00011396408081054688
profit: 358 expected: 358
  time: 0.0007212162017822266
